In [5]:
import yaml

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

seed = config["rnn"]["model"]["seed"]
train_size = config["rnn"]["model"]["train_size"]
val_size = config["rnn"]["model"]["val_size"]
test_size = config["rnn"]["model"]["test_size"]
window_size = config["rnn"]["model"]["window_size"]

In [6]:
# Simulation parameters
N = 5  # Number of particles
timesteps = 10000  # Number of simulation time steps
dt = 1e-2  # Timestep
# alpha = 0.15  # cubic coupling
alphas = [0.1]
beta = 0  # quartic coupling

In [7]:
from functions import Simulation as SIM
from functions import MachineLearning as ML
import os
import numpy as np

q_i = np.sin(np.linspace(0, np.pi*2, N))
p_i = np.zeros(N)

folder_path = 'D:/School/Magistrska/alpha_beta'

sim = SIM(q_i, p_i, N, timesteps, dt)

# Check if the folder is empty
for alpha in alphas:

    qs, ps = sim.integrate(alpha, beta)
    
    sim.save_file(qs, folder_path+str(alpha)+"_"+str(beta)+".npy", alpha, beta)

In [ ]:
qs = np.empty((1, timesteps, N))
alpha_beta = np.empty((1, 2))

for file in os.listdir(folder_path):
    data = np.load(folder_path+"/"+file)
    qs = np.concatenate((qs, data[np.newaxis, 1:]), axis=0)
    alpha_beta = np.concatenate((alpha_beta, data[np.newaxis, 0, :2]), axis=0)

qs = qs[1:]
alpha_beta = alpha_beta[1:]

In [ ]:
ml = ML(qs, ps, window_size, train_size, val_size, test_size)

X, y = ml.make_sequences()

X_train, X_test, X_val, y_train, y_test, y_val = ml.make_split()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

model = Sequential()
model.add(InputLayer((window_size, N)))
model.add(GRU(10, activation="tanh"))
model.add(Dense(2, 'linear'))

model.summary()

In [ ]:
cp = ModelCheckpoint('model/', save_best_only=True)
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), metrics=[RootMeanSquaredError()])

In [ ]:
epochs = 10
batchsize = 256

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batchsize, callbacks=[cp], verbose = 2, workers=-2, use_multiprocessing=True)
from tensorflow.keras.models import load_model
model = load_model('model/')

In [ ]:
test_pred = model.predict(X_test)

In [ ]:
abs = np.linspace(0, 0.6, 5)
indices = []

for alpha in abs:
    for beta in abs:
        current_indices = np.where(np.all(y_test == [alpha, beta], axis=1))[0]
        indices.append([alpha, beta, current_indices])

In [ ]:
import matplotlib.pyplot as plt

for idx in indices:
    # Plot the histogram
    plt.hist(test_pred[idx[2], 0], bins=15, color='r')
    plt.hist(test_pred[idx[2], 1], bins=15, color='g')

    # Add a vertical line
    plt.axvline(x=idx[0], color='r', label='alpha')
    plt.axvline(x=idx[1], color='g', label='beta')

    # Add labels and title
    plt.title("Predicted alphas and betas")
    plt.xticks(rotation=90)

    # Add legend
    plt.legend()

    # Display the plot
    plt.show()
